In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import pickle
from sklearn import preprocessing

In [94]:
pd.set_option('display.max_columns', 500)  # or 1000

In [83]:
datadir = '~/Downloads/datasets-interactive-top-tuples/'
fname = 'house'
fdata = fname + '.csv'

In [100]:
df = pd.read_csv(datadir+fdata, delimiter=',', na_values='NaN')

In [101]:
df[['Type','Region','Municipality','DistrictName','NearestStation','CityPlanning','LandShape','Structure']].isna().sum()

Type                   0
Region            184871
Municipality           0
DistrictName         246
NearestStation      1728
CityPlanning        3826
LandShape         185273
Structure          88218
dtype: int64

In [102]:
df['Type'].unique().shape, df['NearestStation'].unique().shape, df['CityPlanning'].unique().shape, df['DistrictName'].unique().shape, df['Municipality'].unique().shape

((5,), (658,), (17,), (1449,), (59,))

In [103]:
df = df.drop(['No','Region','MunicipalityCode','Prefecture','DistrictName','NearestStation',
              'TimeToNearestStation','LandShape','Structure','Use','Purpose','Direction',
              'Classification','Period','Year','Renovation','Remarks', 'Quarter',
              'UnitPrice', 'PricePerTsubo', 'FloorPlan', 'Frontage', 'TotalFloorArea', 'Breadth' # this line due to NaN
             ],
        axis=1)

In [104]:
df.isna().sum()

Type                               0
Municipality                       0
MinTimeToNearestStation        10339
MaxTimeToNearestStation        10489
TradePrice                         0
Area                               0
AreaIsGreaterFlag                  0
FrontageIsGreaterFlag              0
TotalFloorAreaIsGreaterFlag        0
BuildingYear                   92279
PrewarBuilding                     0
CityPlanning                    3826
CoverageRatio                   5915
FloorAreaRatio                  5915
dtype: int64

In [105]:
df = df.dropna()
df.shape

(303032, 14)

In [106]:
for c in ['Type','Municipality','CityPlanning']:
    df1 = pd.get_dummies(df[c])
    df = pd.concat([df, df1], axis=1)

df = df.drop(['Type','Municipality','CityPlanning'], axis=1)
df.shape

(303032, 77)

In [108]:
df.dtypes.tolist()

[dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('bool'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtyp

In [109]:
np.all((df['Area'] > 0).values)

True

In [110]:
df['UnitPrice'] = df['TradePrice'] / df['Area']
df['BuildingYear'] = 2022 - df['BuildingYear']
df['FrontageIsGreaterFlag'] = df['FrontageIsGreaterFlag'].astype(int)

In [111]:
sum((df['AreaIsGreaterFlag'] > 0).values), sum((df['FrontageIsGreaterFlag'] > 0).values), sum((df['TotalFloorAreaIsGreaterFlag'] > 0).values)

(370, 324, 1168)

In [112]:
df

,MinTimeToNearestStation,MaxTimeToNearestStation,TradePrice,Area,AreaIsGreaterFlag,FrontageIsGreaterFlag,TotalFloorAreaIsGreaterFlag,BuildingYear,PrewarBuilding,CoverageRatio,FloorAreaRatio,"Pre-owned Condominiums, etc.",Residential Land(Land and Building),Adachi Ward,Akiruno City,Akishima City,Arakawa Ward,Bunkyo Ward,Chiyoda Ward,Chofu City,Chuo Ward,Edogawa Ward,Fuchu City,Fussa City,Hachioji City,Hamura City,Higashikurume City,Higashimurayama City,Higashiyamato City,Hino City,"Hinode Town,Nishitama County",Inagi City,Itabashi Ward,Katsushika Ward,Kita Ward,Kiyose City,Kodaira City,Koganei City,Kokubunji City,Komae City,Koto Ward,Kunitachi City,Machida City,Meguro Ward,Minato Ward,Mitaka City,"Mizuho Town,Nishitama County",Musashimurayama City,Musashino City,Nakano Ward,Nerima Ward,Nishitokyo City,Ota Ward,Oume City,Setagaya Ward,Shibuya Ward,Shinagawa Ward,Shinjuku Ward,Suginami Ward,Sumida Ward,Tachikawa City,Taito Ward,Tama City,Toshima Ward,Category I Exclusively Low-story Residential Zone,Category I Exclusively Medium-high Residential Zone,Category I Residential Zone,Category II Exclusively Low-story Residential Zone,Category II Exclusively Medium-high Residential Zone,Category II Residential Zone,Commercial Zone,Exclusively Industrial Zone,Industrial Zone,Neighborhood Commercial Zone,Quasi-industrial Zone,Quasi-residential Zone,Urbanization Control Area,UnitPrice
0,4.0,4.0,40000000,30,0,0,0,9.0,0,80.0,500.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.333333e+06
1,4.0,4.0,130000000,80,0,0,0,15.0,0,80.0,500.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.625000e+06
2,2.0,2.0,400000000,110,0,0,0,4.0,0,80.0,500.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3.636364e+06
3,1.0,1.0,180000000,50,0,0,0,46.0,0,80.0,700.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3.600000e+06
4,4.0,4.0,100000000,65,0,0,0,15.0,0,80.0,500.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.538462e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406451,13.0,13.0,32000000,130,0,0,0,14.0,0,40.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.461538e+05
406452,13.0,13.0,32000000,130,0,0,0,14.0,0,40.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.461538e+05
406454,24.0,24.0,28000000,115,0,0,0,15.0,0,60.0,200.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.434783e+05
406455,16.0,16.0,31000000,130,0,0,0,15.0,0,40.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.384615e+05


In [113]:
names = None
items = df

x = items.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_scaled = x_scaled / norm(x_scaled, axis=1)[:, None]
items = pd.DataFrame(x_scaled, columns=items.columns)

In [114]:
items

,MinTimeToNearestStation,MaxTimeToNearestStation,TradePrice,Area,AreaIsGreaterFlag,FrontageIsGreaterFlag,TotalFloorAreaIsGreaterFlag,BuildingYear,PrewarBuilding,CoverageRatio,FloorAreaRatio,"Pre-owned Condominiums, etc.",Residential Land(Land and Building),Adachi Ward,Akiruno City,Akishima City,Arakawa Ward,Bunkyo Ward,Chiyoda Ward,Chofu City,Chuo Ward,Edogawa Ward,Fuchu City,Fussa City,Hachioji City,Hamura City,Higashikurume City,Higashimurayama City,Higashiyamato City,Hino City,"Hinode Town,Nishitama County",Inagi City,Itabashi Ward,Katsushika Ward,Kita Ward,Kiyose City,Kodaira City,Koganei City,Kokubunji City,Komae City,Koto Ward,Kunitachi City,Machida City,Meguro Ward,Minato Ward,Mitaka City,"Mizuho Town,Nishitama County",Musashimurayama City,Musashino City,Nakano Ward,Nerima Ward,Nishitokyo City,Ota Ward,Oume City,Setagaya Ward,Shibuya Ward,Shinagawa Ward,Shinjuku Ward,Suginami Ward,Sumida Ward,Tachikawa City,Taito Ward,Tama City,Toshima Ward,Category I Exclusively Low-story Residential Zone,Category I Exclusively Medium-high Residential Zone,Category I Residential Zone,Category II Exclusively Low-story Residential Zone,Category II Exclusively Medium-high Residential Zone,Category II Residential Zone,Commercial Zone,Exclusively Industrial Zone,Industrial Zone,Neighborhood Commercial Zone,Quasi-industrial Zone,Quasi-residential Zone,Urbanization Control Area,UnitPrice
0,0.021836,0.016377,0.000322,0.004938,0.0,0.0,0.0,0.045855,0.0,0.491305,0.176870,0.491305,0.000000,0.0,0.0,0.0,0.0,0.0,0.491305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.491305,0.0,0.0,0.0,0.0,0.0,0.0,0.018014
1,0.021775,0.016331,0.001044,0.017234,0.0,0.0,0.0,0.084923,0.0,0.489939,0.176378,0.489939,0.000000,0.0,0.0,0.0,0.0,0.0,0.489939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.489939,0.0,0.0,0.0,0.0,0.0,0.0,0.021894
2,0.010917,0.008188,0.003221,0.024686,0.0,0.0,0.0,0.013100,0.0,0.491257,0.176852,0.000000,0.491257,0.0,0.0,0.0,0.0,0.0,0.491257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.491257,0.0,0.0,0.0,0.0,0.0,0.0,0.049125
3,0.005167,0.003875,0.001372,0.009347,0.0,0.0,0.0,0.272795,0.0,0.464991,0.241795,0.000000,0.464991,0.0,0.0,0.0,0.0,0.0,0.464991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.464991,0.0,0.0,0.0,0.0,0.0,0.0,0.046034
4,0.021777,0.016333,0.000803,0.013542,0.0,0.0,0.0,0.084930,0.0,0.489979,0.176392,0.489979,0.000000,0.0,0.0,0.0,0.0,0.0,0.489979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.489979,0.0,0.0,0.0,0.0,0.0,0.0,0.020730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303027,0.082006,0.061505,0.000298,0.034235,0.0,0.0,0.0,0.090838,0.0,0.113547,0.013626,0.000000,0.567737,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.567737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.567737,0.000000,0.0,0.0,

In [115]:
import os

with open(os.path.expanduser(datadir+fname+'.pkl'), 'wb') as fout:
    pickle.dump([items,names], fout)